In [1]:
import pandas as pd
import os
import math
import time
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import traceback
import cv2
import subprocess
#from moviepy.editor import VideoFileClip
import shutil
from moviepy.video.io.VideoFileClip import VideoFileClip

## Step 1: Separate Left and Right Hand Video Segments

In [2]:
def create_directory(file_path):
    '''
    Create a directory if it does not exist.
    Create parent directories as well if necessary.
    '''
    if os.path.exists(file_path):
        return
    
    parent = os.path.dirname(file_path)
    
    if os.path.exists(parent):
        os.mkdir(file_path)
        return
    
    create_directory(parent)
    os.mkdir(file_path)
    return

def remove_and_clear_directory(file_path):
    shutil.rmtree(file_path)

def number_to_time(time_in_fraction):
    '''
    input: 0.45 = 45 seconds
    output: 00:00:45 (hh:mm:ss)
    '''
    seconds = round(time_in_fraction*100,2)
    assert(seconds<60)
    if seconds<10:
        time = "00:00:0"+str(seconds)
    else:
        time = "00:00:"+str(seconds)
    return time

In [3]:
def video_properties(file_path):
    '''
    Given a file path, returns the following:
    duration (in seconds)
    '''
    vidcapture = cv2.VideoCapture(file_path)
    fps = vidcapture.get(cv2.CAP_PROP_FPS)
    totalNoFrames = vidcapture.get(cv2.CAP_PROP_FRAME_COUNT);
    durationInSeconds = float(totalNoFrames) / float(fps)
    return durationInSeconds

In [4]:
#Read Excel Sheet
INPUT_LR_SPLIT_FILE = "E:/Saiful/park_motor/LeftRight/data/right_left_label.csv"
#Original Task2 Videos
INPUT_TASK2_VIDEOS = "E:/Saiful/park_motor/Task2_All"

remove_and_clear_directory("E:/Saiful/park_motor/LeftRight/Task2_Videos/")

#This is where separated videos will go
OUTPUT_LEFT_VIDEOS = "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left"
OUTPUT_RIGHT_VIDEOS = "E:/Saiful/park_motor/LeftRight/Task2_Videos/Right"

#Create directories (full path if needed) if not present
create_directory(OUTPUT_LEFT_VIDEOS)
create_directory(OUTPUT_RIGHT_VIDEOS)

error_files = []

temp_dir = os.path.join(OUTPUT_LEFT_VIDEOS, "temp")
create_directory(temp_dir)

In [ ]:
success = 0
failed = 0

test_count = 0

df = pd.read_csv(INPUT_LR_SPLIT_FILE)
for index, row in df.iterrows():
    
    filename = row["File_name"]
    left_start = row["left_begin"]
    left_end = row["left_end"]
    right_start = row[" right_begin"]
    right_end = row["right_end"]
    
    print("Left %f to %f, Right %f to %f"%(left_start, left_end, right_start, right_end))
    
    input_file_path = os.path.join(INPUT_TASK2_VIDEOS, filename)
    #Make sure it exists
    assert(os.path.exists(input_file_path))
    
    #Read duration of the video
    #duration = video_properties(input_file_path)
    #print("Duration of webm %f"%(duration))
    
    output_left_filename = filename[:-5]+"-left"+".mp4"
    output_right_filename = filename[:-5]+"-right"+".mp4"
    
    output_left_path = os.path.join(OUTPUT_LEFT_VIDEOS, output_left_filename)
    output_right_path = os.path.join(OUTPUT_RIGHT_VIDEOS, output_right_filename)
    
    
    temp_output_file = filename[:-5]+"-temp"+".mp4"
    temp_path = os.path.join(temp_dir, temp_output_file)
    
    #Resize to 256x256, convert to mp4
    command = "ffmpeg -i \"%s\" -vf scale=256:256 \"%s\""%(input_file_path, temp_path)
    out1 = os.system(command)
    print(out1)
    
    #Read duration of the converted video
    #duration_converted = video_properties(temp_path)
    #print("Duration of resized mp4 %f"%(duration))
    #assert duration==duration_converted
    
    if out1!=0:
        print("MP3 conversion error")
        print(command)
        error_files.append(output_left_path)
        error_files.append(output_right_path)
        failed +=2
    else:
        time.sleep(2)
    #Sample command to cut video by time
    #ffmpeg -i "E:/Saiful/park_motor/Task2_All/2017-08-18T14-59-52-530Z49-task2.webm" -ss 00:00:03 -t 00:00:08 -async 1 -c copy "E:/Saiful/park_motor/LeftRight/Task2_Videos/Test/test.webm"
    
    if out1==0 and (left_start!=(-1.0)) and (left_end!=(-1.0)):
        #Clip left part
        if not os.path.exists(output_left_path):
            #left_start_time = number_to_time(left_start)
            #left_end_time = number_to_time(left_end)
            left_start_time = left_start*100
            left_end_time = (left_end*100)+1
            try:
                with VideoFileClip(temp_path) as video:
                    new = video.subclip(left_start_time, left_end_time)
                    new.write_videofile(output_left_path, audio_codec='aac')
                    
                #ffmpeg_extract_subclip(temp_path, left_start_time, left_end_time, targetname=output_left_path)
                time.sleep(2)
                success +=1
                
            except Exception:
                failed +=1
                error_files.append(output_left_path)
                traceback.print_exc()
            
        
    if out1==0 and (right_start!=(-1.0)) and (right_end!=(-1.0)):
        #Clip right part
        if not os.path.exists(output_right_path):
            #right_start_time = number_to_time(right_start)
            #right_end_time = number_to_time(right_end)
            right_start_time = right_start*100
            right_end_time = (right_end*100)+1
            
            try:
                with VideoFileClip(temp_path) as video:
                    new = video.subclip(right_start_time, right_end_time)
                    new.write_videofile(output_right_path, audio_codec='aac')
                
                #ffmpeg_extract_subclip(temp_path, right_start_time, right_end_time, targetname=output_right_path)
                time.sleep(2)
                success +=1
            except Exception:
                failed +=1
                error_files.append(output_right_path)
                traceback.print_exc()
                
    print("Attempted %d, success %d, failed %d \n"%((failed+success), success, failed))
    
    '''Test code'''
    #test_count +=1
    #if test_count==10:
    #    break

Left 0.020000 to 0.080000, Right 0.080000 to 0.160000
0


t:   0%|                                                                            | 0/7000 [00:00<?, ?it/s, now=None]

Moviepy - Building video E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\2017-08-18T14-59-52-530Z49-task2-left.mp4.
Moviepy - Writing video E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\2017-08-18T14-59-52-530Z49-task2-left.mp4



Moviepy - Done !
Moviepy - video ready E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\2017-08-18T14-59-52-530Z49-task2-left.mp4


t:   0%|                                                                            | 0/9000 [00:00<?, ?it/s, now=None]

...

Moviepy - Building video E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\2017-10-10T21-15-21-095Z89-task2-left.mp4.
Moviepy - Writing video E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\2017-10-10T21-15-21-095Z89-task2-left.mp4



Moviepy - Done !
Moviepy - video ready E:/Saiful/park_motor/LeftRight/Task2_Videos/Left\2017-10-10T21-15-21-095Z89-task2-left.mp4


t:   0%|                                                                            | 0/7000 [00:00<?, ?it/s, now=None]

Moviepy - Building video E:/Saiful/park_motor/LeftRight/Task2_Videos/Right\2017-10-10T21-15-21-095Z89-task2-right.mp4.
Moviepy - Writing video E:/Saiful/park_motor/LeftRight/Task2_Videos/Right\2017-10-10T21-15-21-095Z89-task2-right.mp4



Moviepy - Done !
Moviepy - video ready E:/Saiful/park_motor/LeftRight/Task2_Videos/Right\2017-10-10T21-15-21-095Z89-task2-right.mp4
Attempted 14, success 14, failed 0 

Left 0.030000 to 0.090000, Right 0.000000 to 0.030000
0


In [ ]:
with open('E:/Saiful/park_motor/LeftRight/errors.txt', 'w') as f:
    for item in error_files:
        f.write("%s\n" % item)